Connectome Workbench is required: https://www.humanconnectome.org/software/connectome-workbench

In [ ]:
import numpy as np

birth_age_indices = np.loadtxt('birth_age_full.txt', dtype=str)
scan_age_indices = np.loadtxt('scan_age_full.txt', dtype=str)
all_indices = list(set(birth_age_indices.tolist() + scan_age_indices.tolist()))

all_indices.sort()


In [ ]:
import os

output_path = '../data/features/'

template = True

for index in all_indices:
    sub, ses = index.split('_')
    input_path = '../rel3_dhcp_anat_pipeline/' + sub + '/' + ses
    anat_path = input_path + '/anat/'
    xfm_path = input_path + '/xfm/'
    for hemi in ('left', 'right'):
        myelinmap = anat_path + index + '_hemi-' + hemi + '_myelinmap.shape.gii'
        curv = anat_path + index + '_hemi-' + hemi + '_curv.shape.gii'
        corr_thickness = anat_path + index + '_hemi-' + hemi + '_desc-corr_thickness.shape.gii'
        sulc = anat_path + index + '_hemi-' + hemi + '_sulc.shape.gii'
        metrics = output_path + index + '_' + hemi + '.shape.gii'
        if not os.path.isfile(myelinmap):
            print(myelinmap) # not all T1w images are present in the dataset
        else:
            ico6 = 'ico-6.surf.gii'
            # merge all 4 metrics
            !wb_command -metric-merge $metrics -metric $myelinmap -metric $curv -metric $corr_thickness -metric $sulc
            if template:
                xfm_sphere = xfm_path + index + '_hemi-' + hemi + '_from-native_to-dhcpSym40_dens-32k_mode-sphere.surf.gii'
                template_sphere = 'week-40_hemi-' + hemi + '_space-dhcpSym_dens-32k_sphere.surf.gii'
                native_midthickness = anat_path + index + '_hemi-' + hemi + '_midthickness.surf.gii'
                template_midthickness = 'week-40_hemi-' + hemi + '_space-dhcpSym_dens-32k_midthickness.surf.gii'
                # resample metrics from native space to template sphere (dhcpSym40) using xfm_sphere transform
                !wb_command -metric-resample $metrics $xfm_sphere $template_sphere ADAP_BARY_AREA $metrics -area-surfs $native_midthickness $template_midthickness
                # resample metrics from template sphere (dhcpSym40) to the 6th order icosphere
                !wb_command -metric-resample $metrics $template_sphere $ico6 BARYCENTRIC $metrics
            else:
                native_sphere = anat_path + index + '_hemi-' + hemi + '_sphere.surf.gii'
                # resample metrics from native sphere to the 6th order icosphere
                !wb_command -metric-resample $metrics $native_sphere $ico6 BARYCENTRIC $metrics
    print(index, 'is ready')